### Import Library

In [1]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, accuracy_score

In [2]:
# Path ke folder data utama
data_folder = './data'

# Inisialisasi list kosong untuk menyimpan data gabungan
data_frames = []

In [3]:
# Iterasi melalui setiap folder di direktori data
for label in os.listdir(data_folder):
    label_folder = os.path.join(data_folder, label)

    # Pastikan bahwa ini adalah folder
    if os.path.isdir(label_folder):
        # Iterasi melalui file CSV di dalam folder
        for file in os.listdir(label_folder):
            file_path = os.path.join(label_folder, file)

            # Periksa apakah file tersebut adalah CSV
            if file.endswith('.csv'):
                # Memuat file CSV dan menambahkan label
                temp_data = pd.read_csv(file_path)
                temp_data['label'] = label  # Menetapkan nama folder sebagai label
                data_frames.append(temp_data)

# Menggabungkan semua data menjadi satu DataFrame
df = pd.concat(data_frames, ignore_index=True)

In [4]:
# Memeriksa duplikasi berdasarkan kolom 'Title'
if 'title' in df.columns:
    duplicates = df[df.duplicated(subset='title', keep=False)]  # Melihat data duplikasi untuk ditinjau
    print(f"Jumlah baris duplikat berdasarkan 'title': {len(duplicates)}")

    # Menghapus duplikasi, menyimpan kejadian pertama
    df = df.drop_duplicates(subset='title', keep='first')
    print("Duplikasi dihapus. Baris yang tersisa:", len(df))
else:
    print("Kolom 'title' tidak ada di dataset.")

Jumlah baris duplikat berdasarkan 'title': 26
Duplikasi dihapus. Baris yang tersisa: 23555


In [5]:
# Memeriksa nilai yang hilang atau tidak valid di kolom 'Content'
if 'content' in df.columns:
    print(f"Jumlah nilai yang hilang di 'content': {df['content'].isnull().sum()}")
    df = df.dropna(subset=['content'])  # Menghapus baris dengan 'Content' yang hilang
    df['Content'] = df['content'].astype(str)  # Memastikan semua nilai di 'Content' berupa string
else:
    print("Kolom 'content' tidak ada di dataset.")
    raise ValueError("Kolom 'content' diperlukan untuk klasifikasi teks.")

Jumlah nilai yang hilang di 'content': 916


In [6]:
# Memeriksa tipe data di kolom 'Content'
print("Tipe data di 'Content':", df['content'].apply(type).value_counts())

Tipe data di 'Content': <class 'str'>    22639
Name: content, dtype: int64


In [7]:
# Menghapus baris di mana 'Content' bukan string
df = df[df['content'].apply(lambda x: isinstance(x, str))]
print("Jumlah baris teks valid:", len(df))

# Memeriksa kembali data yang hilang atau tidak sesuai
print("Jumlah nilai yang hilang di 'content':", df['content'].isnull().sum())
print("Contoh baris 'content' yang bermasalah:")
print(df[df['content'].str.strip() == ""].head())

# Menghapus baris dengan 'Content' kosong atau tidak valid
df = df[df['content'].str.strip() != ""]
print("Data telah dibersihkan. Baris yang tersisa:", len(df))
print(df.head())

Jumlah baris teks valid: 22639
Jumlah nilai yang hilang di 'content': 0
Contoh baris 'content' yang bermasalah:
Empty DataFrame
Columns: [link, title, date, content, author, label, Content]
Index: []
Data telah dibersihkan. Baris yang tersisa: 22639
                                                link  \
0  https://www.cnnindonesia.com/ekonomi/202305011...   
1  https://www.cnnindonesia.com/ekonomi/202305011...   
2  https://www.cnnindonesia.com/ekonomi/202305011...   
3  https://www.cnnindonesia.com/ekonomi/202305011...   
4  https://www.cnnindonesia.com/ekonomi/202305011...   

                                               title        date  \
0   Swiss-Belhotel Tuban: Rapat dengan Suasana Re...  2023-05-01   
1   VIDEO: Sambutan Jokowi di Peringatan Hari Bur...  2023-05-01   
2   Cerita Chef Yuli, dari Masak Buat Obama hingg...  2023-05-01   
3   Jokowi: Pemerintah Undang Investasi Demi Kese...  2023-05-01   
4   KAI Catat Arus Balik dari Wilayah Yogyakarta ...  2023-05-01   

    

In [8]:
# Asumsi teks yang akan diklasifikasikan ada di kolom bernama 'Content'
X = df['content']  # Fitur: konten teks
y = df['label']    # Label: nama folder

# Membagi dataset menjadi data training dan testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
# Praproses teks dan ekstraksi fitur menggunakan TF-IDF (stopwords sudah dihapus)
vectorizer = TfidfVectorizer(max_features=1000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [10]:
# Melatih model SVM menggunakan LinearSVC
model = LinearSVC()
model.fit(X_train_tfidf, y_train)

c:\Users\Asus\anaconda3\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


LinearSVC()

In [11]:
print(X_test)

12824    SanhaASTROdikonfirmasi sedang mempersiapkan di...
13200    AktorJoaquin Phoenixhengkang dari film terbaru...
9351     King of the Monsters,Godzilla, akan kembali ke...
17655    kabid binprespp pbsi ricky soebagdja menyebut ...
13454    Bioskop Trans TVmalam ini, Sabtu (7/9), akan m...
                               ...                        
3924     menteri koordinator bidang perekonomianairlang...
1066     otorita kota nusantara iknmenyebutdelapan peru...
11701    Bioskop Trans TVmalam ini, Senin (11/3), akan ...
5779     idul adha muslim disunnahkan membaca takbir or...
9079     Kisah serialXO, Kittybakal berlanjut menuju mu...
Name: content, Length: 4528, dtype: object


In [12]:
# Transformasi X_test menggunakan TF-IDF vectorizer yang sama saat training
X_test_tfidf = vectorizer.transform(X_test)

# Memprediksi menggunakan model yang telah dilatih
y_pred = model.predict(X_test_tfidf)

# Evaluasi model
print("Akurasi:", accuracy_score(y_test, y_pred))
print("Laporan Klasifikasi:\n", classification_report(y_test, y_pred))

Akurasi: 0.9693021201413428
Laporan Klasifikasi:
                precision    recall  f1-score   support

      economy       0.95      0.96      0.95       990
    education       0.96      0.97      0.97       736
entertainment       1.00      1.00      1.00       933
        sport       0.99      0.99      0.99       982
   technology       0.95      0.93      0.94       887

     accuracy                           0.97      4528
    macro avg       0.97      0.97      0.97      4528
 weighted avg       0.97      0.97      0.97      4528



In [13]:
# Memprediksi data baru
new_texts = ["Hasil SEA Games: Duel Alot Banyak Kartu, Kamboja dan Filipina Imbang"]
new_texts_tfidf = vectorizer.transform(new_texts)
predicted_labels = model.predict(new_texts_tfidf)
print("Label Prediksi:", predicted_labels)

Label Prediksi: ['sport']


In [14]:
import joblib

# Menyimpan model
joblib.dump(model, 'text_classification_model.pkl')

# Menyimpan TF-IDF vectorizer
joblib.dump(vectorizer, 'tfidf_vectorizer.pkl')

print("Model dan Vectorizer telah disimpan.")

Model dan Vectorizer telah disimpan.
